# Detecção de COVID-19 em Tomografias — Versão TensorFlow/Keras

Notebook completo pronto para rodar.

**Instruções rápidas**:
- Se estiver no Google Colab, faça upload do `kaggle.json` quando solicitado.
- Ajuste `DATA_DIR` caso já tenha o dataset extraído localmente.
- Execute as células em ordem.

---

## 1) Imports e configurações iniciais

In [ ]:
# Imports e configurações
import os
import random
import numpy as np
import tensorflow as tf

SEED = 42
IMG_SIZE = 224
BATCH_SIZE = 32
VAL_SPLIT = 0.2
EPOCHS = 15

tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

print("TensorFlow:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))

TensorFlow: 2.19.0
GPU available: []


## 2) (Opcional) Instalar e configurar Kaggle API (Colab)

In [ ]:
# Instalar kaggle (execute apenas se precisar baixar do Kaggle via API)
# No Colab, rode esta célula e em seguida faça upload do kaggle.json quando solicitado.
!pip install kaggle

## 3) (Opcional) Baixar e extrair dataset do Kaggle
Dataset usado: plameneduardo/sarscov2-ctscan-dataset

In [ ]:
from google.colab import files
files.upload()  # Envie o arquivo kaggle.json

Saving archive (3).zip to archive (3).zip


In [ ]:
import zipfile

zip_file_name = 'archive (3).zip' # Nome do arquivo ZIP que você fez upload

with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(DATA_DIR) # Extrai para o diretório DATA_DIR

print(f"Conteúdo de {zip_file_name} extraído para {DATA_DIR}/")

Conteúdo de archive (3).zip extraído para dataset/


## 5) Criar splits train/validation de forma segura (equivalente ao random_split do PyTorch)

## 5) Criar splits train/validation de forma segura (equivalente ao random_split do PyTorch)

In [ ]:
from glob import glob
from sklearn.model_selection import train_test_split

def list_images(data_dir):
    exts = ('*.png','*.jpg','*.jpeg','*.bmp')
    files = []
    for ext in exts:
        files.extend(glob(os.path.join(data_dir, '*', ext)))
    return sorted(files)

all_images = list_images(DATA_DIR)
print("Total imagens encontradas:", len(all_images))

train_files, val_files = train_test_split(
    all_images,
    test_size=VAL_SPLIT,
    random_state=SEED,
    shuffle=True
)

print("Train files:", len(train_files), "Val files:", len(val_files))

Total imagens encontradas: 2481
Train files: 1984 Val files: 497


## 6) Data augmentation e funções de pré-processamento

In [ ]:
import tensorflow.keras as keras

# Augmentations (aplicadas apenas no treino)
train_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.05),
    keras.layers.RandomContrast(0.1),
])

def decode_image(path):
    img_bytes = tf.io.read_file(path)

    # Determine file extension to use specific decoder
    file_extension = tf.strings.split(path, '.')[-1]

    # Use tf.case for graph-compatible conditional decoding
    # Each branch must return tensors of the same type and rank.
    def decode_png_branch():
        return tf.image.decode_png(img_bytes, channels=3)

    def decode_jpeg_branch():
        return tf.image.decode_jpeg(img_bytes, channels=3)

    # Fallback for other types or if extension is not perfectly reliable
    def decode_generic_branch():
        return tf.image.decode_image(img_bytes, channels=3)

    img = tf.case([(tf.equal(file_extension, 'png'), decode_png_branch),
                   (tf.logical_or(tf.equal(file_extension, 'jpg'),
                                  tf.equal(file_extension, 'jpeg')), decode_jpeg_branch)],
                  default=decode_generic_branch)

    # Ensure the tensor has 3 dimensions (height, width, channels) after decoding
    # This is crucial for tf.image.resize if its input had ndims=None.
    img = tf.ensure_shape(img, [None, None, 3])

    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    img = tf.cast(img, tf.float32) / 255.0
    return img

# classes list (used in get_label)
classes = sorted([d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))])

def get_label(file_path):
    # Convert os.path.sep to a tf.constant string for graph compatibility
    parts = tf.strings.split(file_path, tf.constant(os.path.sep))
    class_name = parts[-2]
    class_names_tensor = tf.constant(classes)
    eq = tf.equal(class_names_tensor, class_name)
    idx = tf.cast(tf.argmax(tf.cast(eq, tf.int32)), tf.float32) # Cast to float32
    return idx

def load_train(path):
    img = decode_image(path)
    # Removendo temporariamente o data augmentation para isolar o problema
    # img = train_augmentation(img)
    label = get_label(path)
    return img, label

def load_val(path):
    img = decode_image(path)
    label = get_label(path)
    return img, label

## 7) Construir tf.data.Dataset para treino e validação

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(train_files)
train_ds = train_ds.shuffle(1000, seed=SEED)
train_ds = train_ds.map(load_train, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices(val_files)
val_ds = val_ds.map(load_val, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print(train_ds.element_spec)
print(val_ds.element_spec)

(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))
(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))


## 8) Criar modelo (EfficientNetB0 - transfer learning)

In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0

base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = models.Model(inputs, outputs)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,050,852 (15.45 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

## 9) Callbacks e treino

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

checkpoint_path = "best_model_keras.h5"
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)

Epoch 1/15


ValueError: Attr 'Toutput_types' of 'OptionalFromValue' Op passed list of length 0 less than minimum 1.

## 10) Avaliação e métricas (classification report e confusion matrix)

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

y_true = []
y_pred = []

for batch_images, batch_labels in val_ds:
    preds = model.predict(batch_images)
    preds = (preds.flatten() > 0.5).astype(int)
    y_pred.extend(preds.tolist())
    y_true.extend(batch_labels.numpy().tolist())

print(classification_report(y_true, y_pred, target_names=classes))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=classes, yticklabels=classes, cmap='Blues')
plt.xlabel('Predito')
plt.ylabel('Real')
plt.title('Matriz de Confusão')
plt.show()

## 11) Salvar modelo e classes

In [ ]:
model.save('covid_ctscan_model.h5')
import json
with open('class_names.json','w') as f:
    json.dump(classes, f, ensure_ascii=False, indent=2)

print('Modelo salvo em covid_ctscan_model.h5 e classes em class_names.json')

## 12) (Opcional) Grad-CAM para explicabilidade

In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        loss = predictions[:, 0]
    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / (tf.math.reduce_max(heatmap) + 1e-8)
    heatmap = tf.image.resize(heatmap[..., tf.newaxis], (IMG_SIZE, IMG_SIZE))
    heatmap = tf.squeeze(heatmap).numpy()
    return heatmap

# Exemplo de uso
if 'val_files' in globals() and len(val_files) > 0:
    sample_path = val_files[0]
    img = decode_image(sample_path)
    input_array = tf.expand_dims(img, axis=0)
    last_conv_layer_name = None
    for layer in reversed(model.layers):
        if isinstance(layer, tf.keras.layers.Conv2D):
            last_conv_layer_name = layer.name
            break
    if last_conv_layer_name is None:
        print('Não encontrou camada conv no modelo.')
    else:
        heatmap = make_gradcam_heatmap(input_array, model, last_conv_layer_name)
        plt.imshow(img)
        plt.imshow(heatmap, cmap='jet', alpha=0.4)
        plt.axis('off')
        plt.title('Grad-CAM (exemplo)')
        plt.show()

# Task
Modify the `get_label` function in cell `6d5d27f9` to cast the label index to `tf.int32` instead of `tf.float32`, then re-execute cell `6d5d27f9`, cell `9be72733`, and cell `69c55b7c` to recreate the datasets and retrain the model.

## Modify get_label to return int32

### Subtask:
Change the `get_label` function to cast the label index to `tf.int32` instead of `tf.float32`, then re-execute cell `6d5d27f9`.


**Reasoning**:
The subtask requires modifying the `get_label` function to cast the label index to `tf.int32`. I will update the code in cell `6d5d27f9` to reflect this change.



In [ ]:
import tensorflow.keras as keras

# Augmentations (aplicadas apenas no treino)
train_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.05),
    keras.layers.RandomContrast(0.1),
])

def decode_image(path):
    img_bytes = tf.io.read_file(path)

    # Determine file extension to use specific decoder
    file_extension = tf.strings.split(path, '.')[-1]

    # Use tf.case for graph-compatible conditional decoding
    # Each branch must return tensors of the same type and rank.
    def decode_png_branch():
        return tf.image.decode_png(img_bytes, channels=3)

    def decode_jpeg_branch():
        return tf.image.decode_jpeg(img_bytes, channels=3)

    # Fallback for other types or if extension is not perfectly reliable
    def decode_generic_branch():
        return tf.image.decode_image(img_bytes, channels=3)

    img = tf.case([(tf.equal(file_extension, 'png'), decode_png_branch),
                   (tf.logical_or(tf.equal(file_extension, 'jpg'),
                                  tf.equal(file_extension, 'jpeg')), decode_jpeg_branch)],
                  default=decode_generic_branch)

    # Ensure the tensor has 3 dimensions (height, width, channels) after decoding
    # This is crucial for tf.image.resize if its input had ndims=None.
    img = tf.ensure_shape(img, [None, None, 3])

    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    img = tf.cast(img, tf.float32) / 255.0
    return img

# classes list (used in get_label)
classes = sorted([d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))])

def get_label(file_path):
    # Convert os.path.sep to a tf.constant string for graph compatibility
    parts = tf.strings.split(file_path, tf.constant(os.path.sep))
    class_name = parts[-2]
    class_names_tensor = tf.constant(classes)
    eq = tf.equal(class_names_tensor, class_name)
    idx = tf.cast(tf.argmax(tf.cast(eq, tf.int32)), tf.int32) # Cast to int32
    return idx

def load_train(path):
    img = decode_image(path)
    # Removendo temporariamente o data augmentation para isolar o problema
    # img = train_augmentation(img)
    label = get_label(path)
    return img, label

def load_val(path):
    img = decode_image(path)
    label = get_label(path)
    return img, label

**Reasoning**:
The subtask requires re-executing cell `9be72733` to recreate the datasets with the updated `get_label` function that now casts the label index to `tf.int32`.



In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(train_files)
train_ds = train_ds.shuffle(1000, seed=SEED)
train_ds = train_ds.map(load_train, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices(val_files)
val_ds = val_ds.map(load_val, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print(train_ds.element_spec)
print(val_ds.element_spec)

(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))
(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))


In [ ]:
import tensorflow.keras as keras

# Augmentations (aplicadas apenas no treino)
train_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.05),
    keras.layers.RandomContrast(0.1),
])

def decode_image(path):
    img_bytes = tf.io.read_file(path)

    # Determine file extension to use specific decoder
    file_extension = tf.strings.split(path, '.')[-1]

    # Use tf.case for graph-compatible conditional decoding
    # Each branch must return tensors of the same type and rank.
    def decode_png_branch():
        return tf.image.decode_png(img_bytes, channels=3)

    def decode_jpeg_branch():
        return tf.image.decode_jpeg(img_bytes, channels=3)

    # Fallback for other types or if extension is not perfectly reliable
    def decode_generic_branch():
        return tf.image.decode_image(img_bytes, channels=3)

    img = tf.case([(tf.equal(file_extension, 'png'), decode_png_branch),
                   (tf.logical_or(tf.equal(file_extension, 'jpg'),
                                  tf.equal(file_extension, 'jpeg')), decode_jpeg_branch)],
                  default=decode_generic_branch)

    # Ensure the tensor has 3 dimensions (height, width, channels) after decoding
    # This is crucial for tf.image.resize if its input had ndims=None.
    img = tf.ensure_shape(img, [None, None, 3])

    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    img = tf.cast(img, tf.float32) / 255.0
    return img

# classes list (used in get_label)
classes = sorted([d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))])

def get_label(file_path):
    # Convert os.path.sep to a tf.constant string for graph compatibility
    parts = tf.strings.split(file_path, tf.constant(os.path.sep))
    class_name = parts[-2]
    class_names_tensor = tf.constant(classes)
    eq = tf.equal(class_names_tensor, class_name)
    idx = tf.cast(tf.argmax(tf.cast(eq, tf.int32)), tf.int32) # Cast to int32
    return idx

def load_train(path):
    img = decode_image(path)
    # Removendo temporariamente o data augmentation para isolar o problema
    # img = train_augmentation(img)
    label = get_label(path)
    label = tf.expand_dims(label, axis=-1) # Ensure label has shape (1,) for each element
    return img, label

def load_val(path):
    img = decode_image(path)
    label = get_label(path)
    label = tf.expand_dims(label, axis=-1) # Ensure label has shape (1,) for each element
    return img, label

In [ ]:
import tensorflow.keras as keras

# Augmentations (aplicadas apenas no treino)
train_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.05),
    keras.layers.RandomContrast(0.1),
])

def decode_image(path):
    img_bytes = tf.io.read_file(path)

    # Determine file extension to use specific decoder
    file_extension = tf.strings.split(path, '.')[-1]

    # Use tf.case for graph-compatible conditional decoding
    # Each branch must return tensors of the same type and rank.
    def decode_png_branch():
        return tf.image.decode_png(img_bytes, channels=3)

    def decode_jpeg_branch():
        return tf.image.decode_jpeg(img_bytes, channels=3)

    # Fallback for other types or if extension is not perfectly reliable
    def decode_generic_branch():
        return tf.image.decode_image(img_bytes, channels=3)

    img = tf.case([(tf.equal(file_extension, 'png'), decode_png_branch),
                   (tf.logical_or(tf.equal(file_extension, 'jpg'),
                                  tf.equal(file_extension, 'jpeg')), decode_jpeg_branch)],
                  default=decode_generic_branch)

    # Ensure the tensor has 3 dimensions (height, width, channels) after decoding
    # This is crucial for tf.image.resize if its input had ndims=None.
    img = tf.ensure_shape(img, [None, None, 3])

    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    img = tf.cast(img, tf.float32) / 255.0
    return img

# classes list (used in get_label)
classes = sorted([d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))])

def get_label(file_path):
    # Convert os.path.sep to a tf.constant string for graph compatibility
    parts = tf.strings.split(file_path, tf.constant(os.path.sep))
    class_name = parts[-2]
    class_names_tensor = tf.constant(classes)
    eq = tf.equal(class_names_tensor, class_name)
    idx = tf.cast(tf.argmax(tf.cast(eq, tf.int32)), tf.int32) # Cast to int32
    return idx

def load_train(path):
    img = decode_image(path)
    # Removendo temporariamente o data augmentation para isolar o problema
    # img = train_augmentation(img)
    label = get_label(path)
    label = tf.expand_dims(label, axis=-1) # Ensure label has shape (1,) for each element
    return img, label

def load_val(path):
    img = decode_image(path)
    label = get_label(path)
    label = tf.expand_dims(label, axis=-1) # Ensure label has shape (1,) for each element
    return img, label

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(train_files)
train_ds = train_ds.shuffle(1000, seed=SEED)
train_ds = train_ds.map(load_train, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices(val_files)
val_ds = val_ds.map(load_val, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print(train_ds.element_spec)
print(val_ds.element_spec)

(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.int32, name=None))
(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.int32, name=None))


In [ ]:
import os
import random
import numpy as np
import tensorflow as tf

SEED = 42
IMG_SIZE = 224
BATCH_SIZE = 32
VAL_SPLIT = 0.2
EPOCHS = 15
DATA_DIR = 'dataset' # Explicitly define DATA_DIR

tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

print("TensorFlow:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))

TensorFlow: 2.19.0
GPU available: []


In [ ]:
import zipfile

zip_file_name = 'archive (3).zip' # Nome do arquivo ZIP que você fez upload

with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(DATA_DIR) # Extrai para o diretório DATA_DIR

print(f"Conteúdo de {zip_file_name} extraído para {DATA_DIR}/")

Conteúdo de archive (3).zip extraído para dataset/


In [ ]:
from glob import glob
from sklearn.model_selection import train_test_split

def list_images(data_dir):
    exts = ('*.png','*.jpg','*.jpeg','*.bmp')
    files = []
    for ext in exts:
        files.extend(glob(os.path.join(data_dir, '*', ext)))
    return sorted(files)

all_images = list_images(DATA_DIR)
print("Total imagens encontradas:", len(all_images))

train_files, val_files = train_test_split(
    all_images,
    test_size=VAL_SPLIT,
    random_state=SEED,
    shuffle=True
)

print("Train files:", len(train_files), "Val files:", len(val_files))

Total imagens encontradas: 2481
Train files: 1984 Val files: 497


In [ ]:
import tensorflow.keras as keras

# Augmentations (aplicadas apenas no treino)
train_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.05),
    keras.layers.RandomContrast(0.1),
])

def decode_image(path):
    img_bytes = tf.io.read_file(path)

    # Determine file extension to use specific decoder
    file_extension = tf.strings.split(path, '.')[-1]

    # Use tf.case for graph-compatible conditional decoding
    # Each branch must return tensors of the same type and rank.
    def decode_png_branch():
        return tf.image.decode_png(img_bytes, channels=3)

    def decode_jpeg_branch():
        return tf.image.decode_jpeg(img_bytes, channels=3)

    # Fallback for other types or if extension is not perfectly reliable
    def decode_generic_branch():
        return tf.image.decode_image(img_bytes, channels=3)

    img = tf.case([(tf.equal(file_extension, 'png'), decode_png_branch),
                   (tf.logical_or(tf.equal(file_extension, 'jpg'),
                                  tf.equal(file_extension, 'jpeg')), decode_jpeg_branch)],
                  default=decode_generic_branch)

    # Ensure the tensor has 3 dimensions (height, width, channels) after decoding
    # This is crucial for tf.image.resize if its input had ndims=None.
    img = tf.ensure_shape(img, [None, None, 3])

    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    img = tf.cast(img, tf.float32) / 255.0
    return img

# classes list (used in get_label)
classes = sorted([d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))])

def get_label(file_path):
    # Convert os.path.sep to a tf.constant string for graph compatibility
    parts = tf.strings.split(file_path, tf.constant(os.path.sep))
    class_name = parts[-2]
    class_names_tensor = tf.constant(classes)
    eq = tf.equal(class_names_tensor, class_name)
    idx = tf.cast(tf.argmax(tf.cast(eq, tf.int32)), tf.int32) # Cast to int32
    return idx

def load_train(path):
    img = decode_image(path)
    # Removendo temporariamente o data augmentation para isolar o problema
    # img = train_augmentation(img)
    label = get_label(path)
    label = tf.expand_dims(label, axis=-1) # Ensure label has shape (1,) for each element
    return img, label

def load_val(path):
    img = decode_image(path)
    label = get_label(path)
    label = tf.expand_dims(label, axis=-1) # Ensure label has shape (1,) for each element
    return img, label

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(train_files)
train_ds = train_ds.shuffle(1000, seed=SEED)
train_ds = train_ds.map(load_train, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices(val_files)
val_ds = val_ds.map(load_val, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print(train_ds.element_spec)
print(val_ds.element_spec)

(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.int32, name=None))
(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.int32, name=None))


In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0

base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = models.Model(inputs, outputs)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,050,852 (15.45 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

checkpoint_path = "best_model_keras.h5"
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)

Epoch 1/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4780 - loss: 0.7160

62/62 ━━━━━━━━━━━━━━━━━━━━ 213s 3s/step - accuracy: 0.4782 - loss: 0.7158 - val_accuracy: 0.4809 - val_loss: 0.6944 - learning_rate: 0.0010
Epoch 2/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 189s 3s/step - accuracy: 0.5217 - loss: 0.6972 - val_accuracy: 0.4809 - val_loss: 0.6953 - learning_rate: 0.0010
Epoch 3/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5099 - loss: 0.6996

62/62 ━━━━━━━━━━━━━━━━━━━━ 184s 3s/step - accuracy: 0.5098 - loss: 0.6996 - val_accuracy: 0.5191 - val_loss: 0.6924 - learning_rate: 0.0010
Epoch 4/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 188s 3s/step - accuracy: 0.4990 - loss: 0.6994 - val_accuracy: 0.4809 - val_loss: 0.7003 - learning_rate: 0.0010
Epoch 5/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4848 - loss: 0.7002
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
62/62 ━━━━━━━━━━━━━━━━━━━━ 188s 3s/step - accuracy: 0.4847 - loss: 0.7003 - val_accuracy: 0.4809 - val_loss: 0.6952 - learning_rate: 0.0010
Epoch 6/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 243s 4s/step - accuracy: 0.5169 - loss: 0.6987 - val_accuracy: 0.4809 - val_loss: 0.6953 - learning_rate: 5.0000e-04
Epoch 7/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5212 - loss: 0.6937
Epoch 7: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
62/62 ━━━━━━━━━━━━━━━━━━━━ 188s 3s/step - accuracy: 0.5210 - loss: 0.6938 - val_accuracy: 0.4809

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(train_files)
train_ds = train_ds.shuffle(1000, seed=SEED)
train_ds = train_ds.map(load_train, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices(val_files)
val_ds = val_ds.map(load_val, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print(train_ds.element_spec)
print(val_ds.element_spec)

# Inspect a single element from train_ds
print("\nInspecting a single element from train_ds:")
for image, label in train_ds.take(1):
    print(f"  Image shape: {image.shape}, dtype: {image.dtype}")
    print(f"  Label shape: {label.shape}, dtype: {label.dtype}")
    break

# Inspect a single element from val_ds
print("\nInspecting a single element from val_ds:")
for image, label in val_ds.take(1):
    print(f"  Image shape: {image.shape}, dtype: {image.dtype}")
    print(f"  Label shape: {label.shape}, dtype: {label.dtype}")
    break

(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.int32, name=None))
(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.int32, name=None))

Inspecting a single element from train_ds:
  Image shape: (32, 224, 224, 3), dtype: <dtype: 'float32'>
  Label shape: (32, 1), dtype: <dtype: 'int32'>

Inspecting a single element from val_ds:
  Image shape: (32, 224, 224, 3), dtype: <dtype: 'float32'>
  Label shape: (32, 1), dtype: <dtype: 'int32'>
